## SLURM for Beginners
Slurm is software that gives researchers fair allocation of the cluster's resources. It schedules jobs based on the resource requests ( e.g CPUs, RAM, run time, and etc).

### Running SLURM jobs on Cheaha

All work on Cheaha must be submitted to the queueing system, **Slurm**. The purpose of this tutorial is to gives a basic overview of Slurm and guide beginners on how to effectively submit, monitor, and manage jobs on a SLURM-based system. All scripts in this Jupyter Notebook are developed for the **Cheaha**, the UABRC system, so some part of codes/texts may be specific to our setup.

In Cheaha, you can execute your job in two ways:
- **Interactive jobs**: using OOD sessions e.g Desktop, Jupyter Notebook
- **Batch jobs**: can be submitted to the SLURM workload manager using `SBATCH` ( the focus of this tutorial)

To execute a batch job, we use two files:
- the job script ( the actual task script e.g python scripts  with file extensions `.py`)
- the job submission script (`SBATCH` file)

Examples:

Below is the actual job script that prints a simple numpy array and if we run this code block this scipt wil be saved in the same directory as this jupyter notebook as `print_array.py`.

The cells in this Jupyter notebook uses the ipython magic `%%bash` to run the contents of the cell using the bash shell, as though typed at a terminal. The `cat` command concatenates things to a stream. The `>` "file name" means we are redirecting the output of `cat` to the file "file name". The `<<EOT` starts a `heredoc` and redirects it into `cat`. A `heredoc` is what it sounds like, a "fake" file we are making up as we go. Our `heredoc` starts on the line after `cat` and ends on the line before `EOT`. When you plan to write your own code you just ignore these two lines at the begining and the `EOT` at the end. Refer [here](https://github.com/uabrc/How-To-Slurm/blob/main/slurm_how_to.ipynb). Also note that `%%bash` runs an entire cell in bash and `!` runs a single line in bash.

In [ ]:
%%bash
cat <<EOT > "print_array.py"
import numpy as np
import matplotlib
import pylab
import matplotlib.pylab as plt

a = np.array([0, 10, 20, 30, 40])
print(a)

b = np.arange(-5, 5, 0.5)
print(b)

plt.plot([1, 2, 3, 4], [1, 4, 9, 16])
plt.show()
plt.savefig('testing.png')
EOT

In [ ]:
! cat "print_array.py" # just to check if these lines of code are saved in the `print_array.py` file

Below is the job submission script `submit_job_array.sh`, which consists of the a SLURM-based systems job excution stpes:
- Resource request for the job (lines that start with `#SBATCH`)
- Define Job steps:
    - Defines the environment (lines that start with `module` and `conda`)
    - Describe the actual job script (`python print_array.py`)

In [ ]:
%%bash
cat <<EOT > "submit_job_array.sh"
#!/bin/bash
#SBATCH --job-name=even_job       
#SBATCH --output=%x_%j.out           
#SBATCH --error=%x_%j.err            
#SBATCH --partition=amd-hdr100       
#SBATCH --nodes=1                   
#SBATCH --ntasks=1                   
#SBATCH --cpus-per-task=1
#SBATCH --mem=2G
#SBATCH --time=00:20:00

# Load necessary modules
module reset
module load Anaconda3
conda activate tf_gpu
# Run your Python script
python print_array.py
EOT

In [ ]:
!cat "submit_job_array.sh"

### Job  submission 

 Once the actual script and the job submission script are ready, then we can submit the job and monitor the progress of our job. Once we submitted the job, SLURM will first check the job submission script to determine the requested resources and place it on the job queue. Once the resources become available, then the job is executed. See the below examples.

Run the next line to submit the job script to the scheduler.

In [ ]:
!sbatch submit_job_array.sh #submit the job that we defined above

### Monitoring the status of the job

Check the status of your job by using the `squeue` command. The  `squeue` normally used to montior jobs that are currently running and it lists all jobs for a specific researcher.

In [ ]:
!squeue -u $USER

By default the output will show information about the jobs that are include `jobid` as the ID of the job asssigned by the SLURM scheduler,  `partition`, `jobname` as `name`, `blazerid` as `user`, job state as `st`, total run time as `time`, number of nodes as `node`, and the list of nodes as `nodelist` used for each job a researcher has submitted.

Once submitting the job, Slurm will create the output and error files with job name `even_job` and a jobid assigned by the Slurm scheduler (since `even_job` is a job name for this specific example), you can run the `ls` command as given below and see the results. Normally, the ls command will list all files in the directory where we are currently running our job

In [ ]:
!ls # to list files in our directory, and check the slurm output file for results

Once the job you submitted has finished running, use the following two command to check the output and error in the job's log file respectively.

In [ ]:
!cat "even_job_27728525.out"

In [ ]:
!cat "even_job_27728525.err"

### Review results

We can use the `sacct` command to review our previous just for better understanding your own usage history. To read more on `sacct` and reterive customized outputs, please refer our [documentation](https://docs.rc.uab.edu/cheaha/slurm/job_management/#review-jobs-by-jobid).

In [ ]:
!sacct -j [jobid]

### Explore resource usage efficiency

Efficient jobs save you time. Optimizing queue wait times relies on getting resource requests close to actual resource usage. It is important to evaluate the efficiency of your job in terms of resource usage after it completes. `seff` is a tool for exploring resource usage efficiency.

Remember that Cheaha is a shared resource, so requesting resources that sit unused during a job prevents others from using those resources.


In [ ]:
! seff [jobid]

<span style="color:red">**Note**:</span> for running and failed jobs, the efficiency numbers reported by `seff` are not reliable. This tool is usefull only for successfully completed jobs.

### Managing existing job in the queue

Cancelling queued and currently running jobs can be done using the `scancel` command. Importantly, this action will only cancel jobs that were initiated by the researcher running the command. You can run the following commands as per you goal.

In [ ]:
# cancel a single job or an entire job array
!scancel [jobid]

In [ ]:
# cancel all jobs on a partition for the user
scancel -p [partition]

In [ ]:
# cancel all jobs for a researcher
scancel -u $USER

## Submitting Jupyter Notebooks with sbatch on Cheaha

If you would like to submit and run your existing notebook code (in this example, `even.ipynb`), follow these steps:

- Create a job submission script (see the example below) to run the notebook on the Cheaha and save it as a `.sh` file.
- Submit the job to the scheduler using `sbatch`. 

In [ ]:
 # see the next example, assume we have a jupyternotebook "even.ipynb"

#!/bin/bash
#SBATCH --job-name=even_job       
#SBATCH --output=%x_%j.out           
#SBATCH --error=%x_%j.err            
#SBATCH --partition=amd-hdr100       
#SBATCH --nodes=1                   
#SBATCH --ntasks=1                   
#SBATCH --cpus-per-task=1
#SBATCH --mem=2G
#SBATCH --time=00:20:00

# Load necessary modules
module reset
module load Anaconda3
conda activate tf_gpu
#by default, Jupyter Notebook doesn't redirect output to stdout. 
# Only errors and exceptions are typically captured. So, --output=%x_%j.out  will be empty
#cd /path/to/notebook
#save the executed notebook and results as ”even_result.ipynb"
jupyter nbconvert --to notebook --execute even.ipynb --output even_result.ipynb

Then you can browse to the notebook and view the outputs of each cell in Jupyter Notebook as usual.
